<a href="https://colab.research.google.com/github/leotuni/Nicolas-stuff/blob/master/SCTC_leo_no_note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#@title imports
import datetime as dt
import pandas as pd
import pandas_datareader.data as web
import numpy as np 
from itertools import product 


#display tools 
from IPython.display import display
pd.options.display.max_columns = None
# style.use('ggplot') #this is a cool style


In [8]:
#@title Dataframe setup  { form-width: "30px" }
#the date in a nice date format
start_date_data = '2018-03-14' #@param {type:"date"} 
start = dt.datetime.strptime(start_date_data, '%Y-%m-%d') 
stop = dt.datetime.now()

#the symbols you gave me 
symbols = ['BTC-EUR','ETH-EUR', 'LTC-EUR' , 'XRP-EUR', 'BCH-EUR','MIOTA-EUR' ]


#create a dictionary with all the stocks Q: do you prefer to work with lists of dictionaries ? 
dict_of_stock = { s: web.DataReader(s, "yahoo", start, stop) for s in (symbols) }


In [9]:
#@title Functions SMA part { form-width: "30px" }
def conditions_fun (df, col1, col2, choices= '''[df['Close'], -df['Close']]''', defa = 0): 
    ''' 
      dataframe, col2, col2, choice after condition, defa = 0
      returns pandas series, index taken from df 
    
    '''
    conditions = [
                    ((df[col1] > df[col2]) & (df[col1].shift(1) < df[col2].shift(1))),
                    ((df[col1] < df[col2]) & (df[col1].shift(1) > df[col2].shift(1))),
                    ]
    return  (pd.Series(np.select(conditions, choices, default=defa), index=df.index)) #default value we do nothing
 




In [10]:
#@title SMA main { form-width: "30px" }

#Only getting close since it is all we are using 
Close=dict_of_stock['BTC-EUR']['Close']
Close=Close.groupby(Close.index).first() #removes the duplicate indexes

# made them from 2-6 , 7-11 just cause it is easier to see. made into strings cause im using them as col_names
SMA1 = list(map(str, range(2, 6))) #
SMA2 = list(map(str, range(7, 11)))

# # table = [ Close.rolling(s,min_periods=1).mean() for s in (np.arange(1,10))]
data = pd.DataFrame.from_dict({ s: Close.rolling(int(s),min_periods=1).mean() for s in SMA1 + SMA2 }) 
data.insert(0, 'Close', Close)


#This creates a df for the buy and sell values based on the conditions
buy_sell_df = pd.DataFrame.from_dict({  sma1 + '_' + sma2: conditions_fun (data, sma1, sma2 , choices= [data['Close'], -data['Close']])  for sma1, sma2  in product(SMA1, SMA2) })
buy_sell_df.insert(0, 'Close', Close)


# #look at these two 
# data
# buy_sell_df